In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [3]:
dataset = spark.read.csv('/FileStore/tables/hack_data.txt', header= 'True',
                        inferSchema=True)

In [4]:
dataset.head()

Out[6]: Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [5]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [6]:
dataset.columns

Out[9]: ['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [7]:
feat_columns=['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed']

In [8]:
assembler = VectorAssembler(inputCols=feat_columns, outputCol='features')

In [9]:
final_data= assembler.transform(dataset)

In [10]:
final_data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)

In [11]:
from pyspark.ml.feature import StandardScaler

In [12]:
scaler = StandardScaler(inputCol='features', outputCol='Scaaledfeatures' )

In [13]:
scaler_model = scaler.fit(final_data)

In [14]:
scaler_model = scaler.fit(final_data)
cluster_fina_data= scaler_model.transform(final_data)

In [15]:
 cluster_fina_data.printSchema()

root
-- Session_Connection_Time: double (nullable = true)
-- Bytes Transferred: double (nullable = true)
-- Kali_Trace_Used: integer (nullable = true)
-- Servers_Corrupted: double (nullable = true)
-- Pages_Corrupted: double (nullable = true)
-- Location: string (nullable = true)
-- WPM_Typing_Speed: double (nullable = true)
-- features: vector (nullable = true)
-- Scaaledfeatures: vector (nullable = true)

In [16]:
kmeans2= KMeans(featuresCol='Scaaledfeatures', k=2)
kmeans3= KMeans(featuresCol='Scaaledfeatures', k=3)

In [17]:
model_k2 = kmeans2.fit(cluster_fina_data)
model_k3 = kmeans3.fit(cluster_fina_data)

In [18]:
model_k3.transform(cluster_fina_data).groupBy('prediction').count().show()

Exception ignored in: <function JavaWrapper.__del__ at 0x7f2cc1728bf8>
Traceback (most recent call last):
 File "/databricks/spark/python/pyspark/ml/wrapper.py", line 40, in __del__
 if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'StandardScaler' object has no attribute '_java_obj'
+----------+-----+
prediction|count|
+----------+-----+
 1| 83|
 2| 84|
 0| 167|
+----------+-----+

In [19]:
model_k2.transform(cluster_fina_data).groupBy('prediction').count().show()

+----------+-----+
prediction|count|
+----------+-----+
 1| 167|
 0| 167|
+----------+-----+